In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import *
pit_stops_schema = StructType([StructField("driverId", IntegerType(), True), 
                               StructField("duration", StringType(), True), 
                               StructField("lap", IntegerType(), True), 
                               StructField("milliseconds", IntegerType(), True), 
                               StructField("raceId", IntegerType(), True), 
                               StructField("stop", IntegerType(), True), 
                               StructField("time", StringType(), True)])

In [0]:
df = spark.read.option("multiline", "true").schema(pit_stops_schema).json(f"{raw_container_folder_path}/{v_file_date}/pit_stops.json")

In [0]:
from pyspark.sql.functions import current_timestamp,lit
final_df = df.withColumnRenamed("driverId", "driver_id")\
            .withColumnRenamed("raceId","race_id")\
            .withColumn("ingested_date", current_timestamp())\
            .withColumn("data_source",lit(v_data_source))

In [0]:
#final_df.write.mode("overwrite").parquet(f"{processed_container_folder_path}/pit_stops")

#final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed_db.pit_stops")

#load_incremental_data(final_df, "race_id", "f1_processed_db", "pit_stops")

merge_condition = "target_tab.driver_id = source_tab.driver_id AND target_tab.race_id = source_tab.race_id AND target_tab.stop = source_tab.stop AND target_tab.race_id = source_tab.race_id"
merge_incremental_data( final_df, processed_container_folder_path, merge_condition, "race_id", "f1_processed_db", "pit_stops")

In [0]:
dbutils.notebook.exit("success")